In [40]:
#read in emotion file
import pandas as pd
from pd_doc2vec import doc2vec
df1 = pd.read_excel("emotions.xlsx")
print(df1.head())

      ID       Date                                              Tweet  \
0   3020 2017-09-10  @wsvn right below is the parking lot, across f...   
1  67518 2017-09-10     @WPTV #HurricanIrma pic.twitter.com/cpStsW3MXx   
2  20428 2017-09-10  . @DukeEnergy says 447 people without power in...   
3  39812 2017-09-13  So if your regular pickup is on a Tuesday, do ...   
4  60263 2017-09-09  4 more shelters opened in Sarasota Co. bringin...   

  Opinion  Emotion Emotion class Sentiment Sarcasm  
0     NaN      NaN           NaN       NaN     NaN  
1     NaN      NaN           NaN       NaN     NaN  
2     NaN      NaN           NaN       NaN     NaN  
3     NaN      NaN           NaN       NaN     NaN  
4     NaN      NaN           NaN       NaN     NaN  


In [41]:
#prep emotion column for analysis
df1["Emotion"].fillna(0,inplace=True)
print(df1.head())

      ID       Date                                              Tweet  \
0   3020 2017-09-10  @wsvn right below is the parking lot, across f...   
1  67518 2017-09-10     @WPTV #HurricanIrma pic.twitter.com/cpStsW3MXx   
2  20428 2017-09-10  . @DukeEnergy says 447 people without power in...   
3  39812 2017-09-13  So if your regular pickup is on a Tuesday, do ...   
4  60263 2017-09-09  4 more shelters opened in Sarasota Co. bringin...   

  Opinion  Emotion Emotion class Sentiment Sarcasm  
0     NaN      0.0           NaN       NaN     NaN  
1     NaN      0.0           NaN       NaN     NaN  
2     NaN      0.0           NaN       NaN     NaN  
3     NaN      0.0           NaN       NaN     NaN  
4     NaN      0.0           NaN       NaN     NaN  


In [42]:
#trying out Manny's doc2vec class for emotion detection, 10 times
x = doc2vec(df1, "Tweet",["Emotion"], build=True)
print(x.score())

/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Accuracy Score:  0.86
Label Score:  [0.9014084507042254, 0.7586206896551724]
[[0.9014084507042254, 0.7586206896551724], 0.86]


/home/nwest13/Curent/Syd/pd_doc2vec.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test['results'] = self.predict(test[X])


In [ ]:
#TO DO
#try this on other data sets
#repeat for opinion


In [43]:
#dataframe of Emotion ==1 subset
df2 = df.loc[df1["Emotion"] == 1]
print(df2)

Empty DataFrame
Columns: []
Index: []


In [27]:
#Predict Sentiment in hand labeled with emotion
x2 = doc2vec(df2, "Tweet",["Sentiment"], build=True)
print(x.score())

/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Accuracy Score:  0.7
Label Score:  [0.7692307692307692, 0.5714285714285715]
[[0.7692307692307692, 0.5714285714285715], 0.7]


/home/nwest13/Curent/Syd/pd_doc2vec.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test['results'] = self.predict(test[X])


In [31]:
emotion_model = x.model
sentiment_model = x2.model


In [32]:
#connect to mongo db
from pymongo import MongoClient
from pd_doc2vec import doc2vec
import pandas as pd


client = MongoClient('da1.eecs.utk.edu',27017)
print("Connected to Database")

Connected to Database


In [1]:
#NEVER RUN THIS AGAIN. DF IS WHERE ALL COLLECTION DATA IS STORED. LEAVE IT ALONE
#i stole this from manny
from pymongo import MongoClient
from tqdm import tqdm

collection_list = client['twitter'].list_collection_names() 
#Scrapes all the collection names from the Docker
i =0
sentimentdata = []
for posts in (collection_list):
    i = i+1
    print("Processing", posts, i, "/", str(len(collection_list)))
    for post in client['twitter'][posts].find({'sentiment': {"$exists": True}}):  
        #Extracts all the entries with a sentiment field
        post.update({"Source": str(posts)}) 
        #Updates each data with the name of the collection it comes from
        sentimentdata.append(post) 

users = []
collection_list = client['twitter'].collection_names()
collection_list = [x for x in collection_list if "User" in x]
for posts in (collection_list):
    for post in client['twitter'][posts].find():
        users.append(post)

#Scrapes all the collection names from the Docker
i =0
sentimentdata = []
for posts in (collection_list):
    i = i+1
    print("Processing", posts, i, "/", str(len(collection_list)))
    for post in client['twitter'][posts].find({'sentiment': {"$exists": True}}):  
        #Extracts all the entries with a sentiment field
        post.update({"Source": str(posts)}) 
        #Updates each data with the name of the collection it comes from
        sentimentdata.append(post) 

        
        
        
df = pd.DataFrame(sentimentdata)
# pulls data into the Pandas DataFrame

print(df.head(3))

NameError: name 'client' is not defined